In [1]:
import pandas as pd
from kor.extraction import create_extraction_chain
from kor.nodes import Object, Text, Number
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import LlamaCpp
from tqdm import tqdm

Инициализируем LLM

In [2]:
llm = LlamaCpp(
    model_path="../models/Mixtral-8x7B-Instruct-v0.1-GGUF/mixtral-8x7b-instruct-v0.1.Q6_K.gguf",
    n_gpu_layers=50,
    n_batch=512,
    n_ctx=3500,
    f16_kv=True,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    verbose=True,
    temperature = 0
)

llama_model_loader: loaded meta data with 26 key-value pairs and 995 tensors from ../models/Mixtral-8x7B-Instruct-v0.1-GGUF/mixtral-8x7b-instruct-v0.1.Q6_K.gguf (version GGUF V3 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q6_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:          blk.0.ffn_gate.0.weight q6_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    2:          blk.0.ffn_down.0.weight q6_K     [ 14336,  4096,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_up.0.weight q6_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    4:          blk.0.ffn_gate.1.weight q6_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    5:          blk.0.ffn_down.1.weight q6_K     [ 14336,  4096,     1,     1 ]
llama_model_loader: - tensor    6:            blk.0.ffn_up.1.weight q6_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    7:          blk.0.ffn_ga

In [4]:
df = pd.read_csv('./data/data_merge.csv')
df['part_1'] = [x.split('II')[0] for x in df['text']]

Схема для выгрузки данных из applic, раздел I:
* локация
* описание тендера
* название тендера
* идентификатор тендера

In [5]:
schema = Object(
    id="tender_info",
    description="common tender information.",
    attributes=[
        Text(
            id="location",
            description="The territory where the tender project is implemented, which is important for determining jurisdiction and local conditions.",
            examples=[("Территория, в которой осуществляется тендерный проект: Гуанси-Маньчжурский автономный округ, Хочжоу, Чжуншань", "Гуанси-Маньчжурский автономный округ, Хочжоу, Чжуншань")],
        ),
        Text(
            id="product",
            description="Product or service to be provided.",
            examples=[("Условия торгов Проект строительства новой станции HLF", "Проект строительства новой станции HLF")],
        ),
        Text(
            id="tender_number",
            description="Number of tender",
            examples=[("номер тендера: CGN-20230324006","CGN-20230324006")]
        ),
        Text(
            id="tender_name",
            description="Name of tender",
            examples=[("Китайский документ 51-13trans.txt Объявление тендера на строительство новой станции HLF","Объявление тендера на строительство новой станции HLF")]
        ),
        Text(
            id="tender_company",
            description="The name of the company organizing the tender.",
            examples=[("организатором конкурса является компания &lt; &lt; Чайна Хинэ &gt; &gt;", "Чайна Хинэ"), ("участие в тендере компании &quot; Чайна Хинси &quot;", "Чайна Хинси")]
        ),
    ],
    examples=[
        (
            """20230731_12345
            Китайский документ 14-2trans.txt
            Проект SK LOT101Df Архипелагические клапаны и системы REN
            (Контингент: CGN-20230619005-N1)
            - район, в котором осуществляется тендерный проект: провинция Шаньдун;
            Условия торгов
            SKLOT101DfRENCGN-20230619005-N1
            Утверждены клапаны системы &lt; &lt; Ядерный остров &gt; &gt; и &lt; &lt; Система &gt; &gt; (номер тендера:
            Источником финансирования является самофинансируемое предприятие, которое было привлечено к участию в конкурсе на участие в конкурсе компании &quot; Чайна инжиниринг лтд. &quot; . В рамках этого проекта уже имеются условия для проведения торгов и в настоящее время проводятся открытые торги.
            """,
            [
                {"location": "провинция Шаньдун", "product": 'клапаны системы',"tender_number":"CGN-20230619005-N1",
                 "tender_name": "Архипелагические клапаны и системы REN", "tender_company": "Чайна инжиниринг лтд"},
            ],
        ),
        (
            """
            20230731_12345
            Китайский документ 85-8trans.txt
            Глава I Уведомление о торгах
            Район, в котором осуществляется тендерный проект: Немонгольский автономный район, УНИТА, правый флаг Корнея
            Условия торгов
            В рамках этого проекта &lt; &lt; Внутренняя Монголия &gt; &gt; закупила оборудование для оптических камер УНИТА (первый пункт: C, D) (заявочный номер проекта: CGN-
            202211140006 Утверждено Уполномоченным фондом, финансируемым за счет средств, полученных от предприятий, и реализовано, а организатором конкурса является Китай.
            Новая энергетическая компания Corp. В рамках этого проекта уже имеются условия для проведения торгов и в настоящее время проводятся открытые торги.
            """,
            [
                {"location": "Немонгольский автономный район, УНИТА, правый флаг Корнея", "product": 'оборудование для оптических камер',"tender_number":"CGN-202211140006",
                 "tender_name": "", "tender_company": "Новая энергетическая компания"},
            ],
        ),
        (
            """
            20230731_12345
            Китайский документ 28-2trans.txt
            Глава I. Объявление о торгах на приобретение динамической системы многопрофильной загрузки
            (Контингент: CGN-202305230007)
            Место проведения торгов: Гуандун
            Условия торгов
            Проект Закупка динамической системы многопрофильной рефлексии (закупка тендера No CGN-20230523007) завершена
            Проверено, проект финансируется за счет собственных средств и реализован, а тендер - за счет компании China Institute of Nuclear. В рамках этого проекта уже имеется тендерная заявка
            В настоящее время проводятся открытые торги.
            """,
            [
                {"location": "Гуандун", "product": 'Закупка динамической системы многопрофильной рефлексии',"tender_number":"CGN-202305230007",
                 "tender_name": "Объявление о торгах на приобретение динамической системы многопрофильной загрузки", "tender_company": "China Institute of Nuclear"},             
            ]
        )
    ],
    many=True,
)

создаем объект chain с указание формата возвращаемых данных

In [ ]:
chain = create_extraction_chain(llm, schema, encoder_or_encoder_class='json')

через цикл обращаемся к llm

In [ ]:
result = []
for i_text in tqdm(df['part_1']):
    ans = chain.run(text=(i_text))
    result.append(ans['data'])

In [ ]:
temp_list = []
for i in tqdm(range(len(result))):
    if len(result[i]) > 0:
        temp_list.append(result[i]['tender_info'][0])
    else:
        temp_list.append({'location': '',
                        'product': '',
                        'tender_number': '',
                        'tender_name': '',
                        'tender_company': ''})

In [ ]:
temp_df = pd.DataFrame(temp_list)

In [ ]:
temp_df

In [ ]:
# temp_df.to_csv('./data/part_1_BIG_MODELfor_merge.csv', index=False)

In [6]:
df['part_2'] = [x.split('III')[0] for x in df['text']]
df['part_2'] = [x.split('II')[-1] for x in df['part_2']]

создаем схему для раздела II:
* описание предмета тендера
* требования к заявителю
* этапы реализации заявки
* период поставки товара или оказания услуг после заключения договора
* дата поставки товара или оказания услуг
* дата завершения поставки или оказания услуг

In [7]:
schema = Object(
    id="project_info",
    description="detailed information about project.",
    attributes=[
        Text(
            id="description",
            description="Detailed description of project.",
            examples=[("1. Масштаб проекта: проект расположен на экспериментальном заводе No 2 на промышленной базе компании &lt; &lt; Хинон-Юнк &gt; &gt; в Чжуншань, провинция Гуандун\n Внутри. Проект строительства многосвободной системы динамического подъема требует строгой калибровки антиядерной базы и определения структурно-конструкторских составов. Дело, в котором определены требования к строительным процессам, техническая форма, например, проектная программа и проектирование архитектурных чертежей, необходимых для этого проекта.\n2004/2005 год Этот проект закупок включает два этапа проектной работы, первый этап которой начинается после подписания контракта и второй этап - с участием участников торгов.", 
                       "Проект строительства многосвободной системы динамического подъема требует строгой калибровки антиядерной базы и определения структурно-конструкторских составов.\n Дело, в котором определены требования к строительным процессам, техническая форма, например, проектная программа и проектирование архитектурных чертежей, необходимых для этого проекта.\n2004/2005 год Этот проект закупок включает два этапа проектной работы, первый этап которой начинается после подписания контракта и второй этап - с участием участников торгов.")],
        ),
        Text(
            id="requirements",
            description="Technical and design requirements of project.",
            examples=[("Содержание и сфера охвата запроса:\n1) Реакционная базовая программа и проектирование чертежей.\n2) Техническое руководство по проектированию чертежей и фундаментальному строительству на месте.", 
                       "1) Реакционная базовая программа и проектирование чертежей.\n2) Техническое руководство по проектированию чертежей и фундаментальному строительству на месте.")],
        ),
        Text(
            id="stages",
            description="Project stages and timelines.",
            examples=[("Этот проект закупок включает два этапа проектной работы, первый этап которой начинается после подписания контракта и второй этап - с участием участников торгов.",
                       "2"),
                       ("данный тендерный проект разделен на 1 пункт", "1"), 
                       ("Данный тендерный проект делится на четыре сегмента:", '4')
                       ]
        ),
        Text(
            id="delivery_time",
            description="Product delivery time.",
            examples=[("Срок поставки: срок поставки в течение 45 рабочих дней, в зависимости от того, был ли заключен контракт с участником торгов","45"), 
                      ("Дата запланированной поставки: ожидаемая от покупателя дата поставки составляется на месяц.", "30"),
                      ("Планируемый период\n78 календарных дней работы", "78")
                      ]
        ),
        Text(
            id="start_date",
            description="Start date of work.",
            examples=[("Дата начала работ 15 мая 2023 года\nДата завершения проекта 31 июля 2023 года", "15.05.2023"), 
                      ("Дата запланированной поставки: 2023-04-30 (первая партия)", "30.04.2023"),
                      ("Дата запланированной поставки: первая партия: BZ: 2024-07-30; AD: 31 мая 2024 года.", "30.07.2024"),
                      ("Проект &lt; &lt; Старый ветер в Синьцзян-Тауне &gt; &gt; : 20 марта 2023 года (в соответствии с письменным уведомлением министерства)", "20.03.2023"),
                      ("Дата завершения проекта: 30 октября 2023 года", "30.10.2023")
                      ]
        ),
        Text(
            id="end_date",
            description="End date of work.",
            examples=[("Дата начала работ 15 мая 2023 года\nДата завершения проекта 31 июля 2023 года", "31.06.2023"), 
                      ]
        ),
    ],
    examples=[
        (
            """
            . ОБЩИЙ ОБЗОР ПРОЕКТОВ И ОГРАНИЧЕНИЙ
            Описание проекта: Закупка потребностей в исследованиях и разработках для проекта экспериментальной платформы электрической реле для защиты электрической реле, которая в основном предусматривала проведение научно-исследовательского проекта
            Программа включает общие требования к проектированию, материалам, изготовлению оборудования, проверке, испытанию, идентификации, оценке и контролю за документацией. Услуги или предметы
            Поставщики (далее "В"), помимо требований настоящего Технического регламента, должны отвечать существующим стандартам в стране и отрасли.
            Соответствующие положения.
            Содержание и сфера охвата тендера: в ходе конкурса было разработано 7 сканеров и комплектов для сканирования аптечек, шкафов и т.д., а также предложено
            Услуги по установке и отладке аппаратного обеспечения.
            Единица измерения количества серийных номеров
            Анализ потребностей в оборудовании 1
            2 аппаратных платформы для разработки 1
            1 шкаф для проверки 3 локомотивов и 1 шкаф для защиты маршрутов
            Оборудование для переключателя переключателя, защищенные карты
            14 единиц
            шкаф
            5 защитных карточек 1 шкаф
            1 шкаф для 6 карт защиты давления
            1 шкаф для 7-дюймовых магнитных карт
            8 универсальных реле 1 шкаф
            1 шкаф для 9 универсальных трансформаторов
            10 программных интерфейсов для разработки 1
            11 тестов для 1
            12 Установка и отладка.
            1
            13 документов, написанных сверху
            Дата запланированной поставки: 15 августа 2023 года - 31 декабря 2023 года.
            Общий срок службы: 138 календарных дней.
            """,
            [
                {"description": 'Закупка потребностей в исследованиях и разработках для проекта экспериментальной платформы электрической реле для защиты электрической реле, которая в основном предусматривала проведение научно-исследовательского проекта\nПрограмма включает общие требования к проектированию, материалам, изготовлению оборудования, проверке, испытанию, идентификации, оценке и контролю за документацией. Услуги или предметы\nПоставщики (далее "В"), помимо требований настоящего Технического регламента, должны отвечать существующим стандартам в стране и отрасли.', 
                 "requirements": 'в ходе конкурса было разработано 7 сканеров и комплектов для сканирования аптечек, шкафов и т.д., а также предложено\nУслуги по установке и отладке аппаратного обеспечения.',
                 "stages":"1",
                 "delivery_time": "138",
                 "start_date": "15.08.2023",
                 "end_date": "31.12.2023",
                 },
            ],
        ),
    ],
    many=True,
)

In [ ]:
chain = create_extraction_chain(llm, schema, encoder_or_encoder_class='json')

In [ ]:
result = []
for i_text in tqdm(df['part_2'][325:]):
    try:
        ans = chain.run(text=(i_text))
        result.append(ans['data'])
    except:
        result.append({'project_info': [ {"description": '', 
                                           "requirements": '',
                                           "stages":"",
                                           "delivery_time": "",
                                           "start_date": "",
                                           "end_date": "",}]}),

In [ ]:
temp_list = []
for i in tqdm(range(len(result))):
    try:
        if len(result[i]) > 0:
            temp_list.append(result[i]['project_info'][0])
        else:
            temp_list.append({'description': '',
                            'requirements': '',
                            'stages': '',
                            'delivery_time': '',
                            'start_date': '',
                            'end_date': ''})
    except:
        temp_list.append({'description': '',
                'requirements': '',
                'stages': '',
                'delivery_time': '',
                'start_date': '',
                'end_date': ''})

In [ ]:
temp_df = pd.DataFrame(temp_list)

In [ ]:
temp_df

In [ ]:
# temp_df.to_csv('./data/part_2_for_merge.csv', index=False)

In [8]:
df['part_3'] = [x.split('III')[-1] for x in df['text']]

Схема для третьего и последующих разделов applic:
* Информация о базовой квалификации кандидата
* Информация о квалификационных требованиях
* Финансовые требования
* Требования к произволственным мощностям
* Требования к финансовой репутации
* Дата начала тендера
* Дата окончания тендера

In [9]:
schema = Object(
    id="application_data",
    description="Tender application data",
    attributes=[
        Text(
          id='basic_qualification',
          description="Basic candidate qualifications",
          examples=[("""участник процедур, зарегистрированный на территории Китайской Народной Республики в соответствии с Законом Китайской Народной Республики о компаниях, имеет право
            Законодательная власть, предприятия, которые могут предложить продукты и услуги для проекта.""",
                    """участник процедур должен быть зарегистрирован на территории Китайской Народной Республики в соответствии с Законом Китайской Народной Республики о компаниях
            Независимые юридические лица, предприятия, способные предлагать продукты и услуги для проекта.""")],
            ),
        Text(
          id='qualification_requirements',
          description="Qualification requirements",
          examples=[("""Генеральный контрактный уровень (или уровень А) и выше в области электротехники.""",
                    """Для участия в торгах требуется, чтобы участники торгов обладали такими качествами, как общий контракт на строительство гидроэлектроэнергии и выше. (если в соответствии с
Министерство по вопросам городского и сельского строительства в связи с выпуском циркуляра о программе реформы системы управления инженерно-строительными предприятиями в целях повышения квалификации или новых сертификатов для гидроэлектроэнергетики
Генеральный контракт на строительство уровня А и выше).""")],
            ),
        Text(
          id='financial_requirements',
          description="Financial requirements",
          examples=[("""Финансовые требования: аудиторские заключения и финансовые ведомости, представленные зарегистрированной аудиторской фирмой за последние три года (2020-2022 годы)
балансовая ведомость, ведомость прибылей, ведомость движения денежной наличности и приложение.""",
                    """Отчет о финансовой ревизии за последние три года (2019-2021).""")],
            ),
        Text(
          id='performance_requirements',
          description="Performance requirements",
          examples=[("""должны иметь историю и результативность как минимум на три года производства товаров или аналогичных товаров. (Оборудование для замены коробки должно быть
            Комбинированная или сборная с емкостью 3150 КВА и выше, электрическое давление на 35 КВ и выше.""",
                    """в течение почти трех лет (с момента подачи тендерной документации, по крайней мере, одного крана и поставки
Установка работы.""")],
            ),
        Text(
          id='credit_requirements',
          description="Credit requirements",
          examples=[(""" участники торгов имеют хорошую банковскую и коммерческую репутацию, не находятся в ситуации, когда им предписывается прекратить работу, захват имущества, замораживание
            Конкретно, несостоятельно и не находится в пределах и в течение срока, в течение которого компания &lt; &lt; Чайна холдинг компани лимитед &gt; &gt; ограничила свои предложения.""",
                    """не включен в систему раскрытия информации о государственных предприятиях (www.gsxt.gov.cn)
                      В список и на веб-сайте «Доверие в Китай» (http://www.creditchina.gov.cn/) не было внесено в список «Посланник с неверием».""")],
            ),
        Text(
          id='tender_start_date',
          description="Start date for receiving tender documentation",
          examples=[("""07 июня 2023 года, 17:00""",
                    """28 декабря 2022 года в 17 ч. 00 м.""")],
            ),
        Text(
          id='tender_end_date',
          description="End date for receiving tender documentation",
          examples=[("""16 июня 2023 года, 17:00""",
                    """06 января 2023 года 17 ч. 30 м.""")],
            ),
    ],
    many=False,
)


In [ ]:
chain = create_extraction_chain(llm, schema,encoder_or_encoder_class='json')

In [ ]:
result = []
for i_text in tqdm(df['part_3']):
    try:
        ans = chain.run(text=(i_text))
        result.append(ans['data'])
    except:
        result.append({'basic_qualification': '', 'qualification_requirements': '',
   'financial_requirements': '', 'performance_requirements': '', 'credit_requirements': '', 
   'tender_start_date': '',
   'tender_end_date': '',})

In [ ]:
temp_list = []
for i in tqdm(range(len(result))):
    if len(result[i]) > 0:
        temp_list.append(result[i]['application_data'])

In [ ]:
temp_df = pd.DataFrame(temp_list)

In [ ]:
# temp_df.to_csv('./data/part3_for_merge_PART.csv', index=False)

 Схема для текстов из meta:
* Дата заявки
* Идентификатор тендера
* Краткое описание
* Подробное описание 
* Тип исполнителя
* Имя исполнителя 
* Валюта договора
* Наименование заказчика
* Тип заказчика
* Агент

In [ ]:
schema = Object(
    id="meta_data",
    description="Tender meta data",
    attributes=[
        Text(
          id='release_date',
          description="Tender release date",
          examples=[("2023-06-27", "2023-04-11")],
        ),
        Text(
          id='CGN',
          description="CGN of the tender",
          examples=[("CGN-202304240017", "CGN-20180529004")],
        ),
        Text(
            id="brief_description",
            description="Brief description of the tender task",
            examples=[("""Дороги, платформы для подвески, фундаментальные работы в ветряных машинах
                            Результаты в трех пунктах""", 
                       "Установлены результаты первого из трех башен в морской ветряной трубе на острове Янцзы")],
        ),
        Text(
            id="detailed_description",
            description="Detailed description of the tender task",
            examples=[("""Тюрьма Си, Хинпхендо, чайный фонтан и электрическая дорога в рамках проекта Ветер , платформа для подвески и три этапа базовой инженерной работы ветроустановок (прибор)""", 
                       "Первый три башни проекта морской ветровой энергетики на Кананге, Китай, и Яньцзян")],
        ),
        Text(
            id="executor_type",
            description="Type of tenderer company",
            examples=[("Производитель", "Победитель")],
        ),
        Text(
            id="executor_name",
            description="Name of the company executing the tender",
            examples=[("China 19 Metals Group", "Hongong Kong")],
        ),
        Text(
            id="price_currency",
            description="Currency of the tender price",
            examples=[("¥","RMB")]
        ),
        Text(
            id="customer_name",
            description="Name of company customer of the tender",
            examples=[("Ветроэнергетика, Чайна", "Чайна инжиниринг лимитед")],
        ),
        Text(
            id="customer_type",
            description="Type of tender customer company",
            examples=[("Участники торгов", "Заявители")],
        ),  
        Text(
            id="bidding_agent",
            description="Bidding Agent",
            examples=[("Чайна инжиниринг лимитед", "Чайна инжиниринг лимитед")],
        ), 
    ],
    examples=[
        (
            """
            Дата выхода: 2023-06-27
            Дороги, платформы для подвески, фундаментальные работы в ветряных машинах
            Результаты в трех пунктах
            (Контингент: CGN-202304240017)
            Тюрьма Си, Хинпхендо, чайный фонтан и электрическая дорога в рамках проекта &lt; &lt; Ветер &gt; &gt; , платформа для подвески и три этапа базовой инженерной работы ветроустановок (прибор)
            Номер проекта: CGN-202304240017, идентифицированные лица:
            1. Информация о кандидатах:
            Производитель: China 19 Metals Group
            Цены: ¥33 750 545.00
            II. ДРУГИЕ ОБЯЗАТЕЛЬСТВА:
            Нет
            III. УПРАВЛЕНИЕ ВЕРХОВНОГО КОМИССАРА
            Надзор за осуществлением данного тендерного проекта /.
            IV. Связи
            Участники торгов: &lt; &lt; Ветроэнергетика &gt; &gt; , &lt; &lt; Чайна &gt; &gt;
            Контактное лицо: джентльмен
            Тел.: 010-63711966
            Электронная почта: guobijun@cgnpc.com
            Агент по торгам: &quot; Чайна инжиниринг лимитед &quot;
            Контактное лицо: Ли Хун
            Тел.: 0755-84436511
            Электронная почта:
            lhyts@cgnpc.com.cn
            Участник торгов или его главный руководитель (руководитель проекта): (подпись)
            Участник торгов или его агент: (глава)
            """,
            [
                {'release_date':"2023-06-27", 
                 'CGN':"CGN-202304240017", 
                 "brief_description":"""Дороги, платформы для подвески, фундаментальные работы в ветряных машинах
Результаты в трех пунктах""", 
                 "detailed_description":"""Тюрьма Си, Хинпхендо, чайный фонтан и электрическая дорога в рамках проекта Ветер, платформа для подвески и три этапа базовой инженерной работы ветроустановок (прибор)""", 
                 "executor_type":"Производитель", 
                 "executor_name":"China 19 Metals Group", 
                 "price_currency":"¥",
                 "customer_name":"Ветроэнергетика, Чайна", 
                 "customer_type":"Участники торгов", 
                 "bidding_agent":"Чайна инжиниринг лимитед"}
            ],
        ),
        (
            """
            Дата выхода: 2023-04-11
            Установлены результаты первого из трех башен в морской ветряной трубе на острове Янцзы
            (Контингент: CGN-20180529004)
            Первый три башни проекта морской ветровой энергетики на Кананге, Китай, и Яньцзян (заявление No CGN-20180529004)
            Установлено следующее:
            1. Информация о кандидатах:
            Hongong Kong
            Победитель:
            Цены: RMB14 068 380.00
            II. ДРУГИЕ ОБЯЗАТЕЛЬСТВА:
            III. УПРАВЛЕНИЕ ВЕРХОВНОГО КОМИССАРА
            .
            Наблюдение за осуществлением данного тендерного проекта /
            IV. Связи
            Заявители: &quot; Чайна инжиниринг лимитед &quot;
            Местонахождение:/
            Организация: двойная
            Телефон: 0755-88616541
            Электронная почта: luozhenggang@cgnpc.com.cn
            Агент по торгам: &quot; Чайна инжиниринг лимитед &quot;
            Местонахождение:/
            Организация: Худо
            Телефон: 0755-88610728
            Электронная почта:huduo@cgnpc.com.cn
            Участник торгов или его главный руководитель (руководитель проекта): (подпись)
            Участник торгов или его агент: (глава)
            """,
            [
                {'release_date':"2023-04-11", 
                 'CGN':"CGN-20180529004", 
                 "brief_description":"""Установлены результаты первого из трех башен в морской ветряной трубе на острове Янцзы""", 
                 "detailed_description":"""Первый три башни проекта морской ветровой энергетики на Кананге, Китай, и Яньцзян""", 
                 "executor_type":"Победитель", 
                 "executor_name":"Hongong Kong", 
                 "price_currency":"RMB",
                 "customer_name":"Чайна инжиниринг лимитед", 
                 "customer_type":"Заявители", 
                 "bidding_agent":"Чайна инжиниринг лимитед"}
            ],
        ),
    ],
    many=True,
)

In [ ]:
chain = create_extraction_chain(llm, schema,encoder_or_encoder_class='json')

In [ ]:
result = []
for i_text in tqdm(df['meta']):
    ans = chain.run(text=(i_text))
    result.append(ans['data'])

In [ ]:
temp_list = []
for i in tqdm(range(len(result))):
    if len(result[i]) > 0:
        temp_list.append(result[i]['meta_data'][0])
    else:
     temp_list.append({'release_date':"", 
                 'CGN':"", 
                 "brief_description":"", 
                 "detailed_description":"", 
                 "executor_type":"", 
                 "executor_name":"", 
                 "price_currency":"",
                 "customer_name":"", 
                 "customer_type":"", 
                 "bidding_agent":""})

In [ ]:
temp_df = pd.DataFrame(temp_list)

In [ ]:
temp_df